### This program will melt the  preprocessed! calendar and sales files into a long table with all the original and encoded columns

#### Requires extensive CPU, Memory, and Time Resources.  
#### Overrides data in the same directory. 
#### Back up data in the temp and data folders prior to running

____________________________________________________

## Import the standard libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
import sklearn as sk
import category_encoders as ce
import statsmodels as sm
import pdb

In [2]:
sales_df = pd.read_csv('temp/M5_preprocessed_sales-v1.csv', index_col=0)
calendar_df = pd.read_csv('temp/M5_preprocessed_calendar-v1.csv', index_col=0, parse_dates=['date'])
prices_df = pd.read_csv('data/sell_prices.csv')

In [3]:
# Review the sales data
sales_df.head()

,id,item_id_#,item_id,dept_id,cat_id,store_id,state_id,dept_id_1,dept_id_2,dept_id_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,2,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,3,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,4,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,5,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,0,...,2,1,1,0,1,1,2,2,2,4


In [4]:
# Note the first and the last day columns
first_day = 'd_1'
last_day = 'd_1913'

In [5]:
#columns to melt start with day 1: 'd_1' column and end with 'd_1913'
columns_sales = list(sales_df.columns[sales_df.columns.get_loc('d_1'):])

## Reorganize the columns and finalize the Data Schema

In [6]:
#Review the calendar data
len(calendar_df.columns)

49

In [7]:
calendar_df.columns

Index(['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'day', 'Halloween', 'Christmas',
       'EidAlAdha', 'Eid al-Fitr', 'OrthodoxChristmas', 'NewYear',
       'Mother's day', 'ValentinesDay', 'Father's day', 'Thanksgiving',
       'StPatricksDay', 'IndependenceDay', 'MemorialDay', 'PresidentsDay',
       'MartinLutherKingDay', 'VeteransDay', 'LaborDay', 'ColumbusDay',
       'LentStart', 'LentWeek2', 'OrthodoxEaster', 'Chanukah End',
       'Ramadan starts', 'Easter', 'Cinco De Mayo', 'Pesach End', 'Purim End',
       'NBAFinalsStart', 'NBAFinalsEnd', 'SuperBowl', 'Cultural', 'Religious',
       'Sporting', 'National'],
      dtype='object')

In [8]:
#Review the Sales columns upto the daily sales
len(sales_df.columns[:sales_df.columns.get_loc('d_1')])

43

In [9]:
sales_df.columns[:sales_df.columns.get_loc('d_1')]

Index(['id', 'item_id_#', 'item_id', 'dept_id', 'cat_id', 'store_id',
       'state_id', 'dept_id_1', 'dept_id_2', 'dept_id_3', 'dept_id_4',
       'dept_id_5', 'dept_id_6', 'dept_id_7', 'cat_id_1', 'cat_id_2',
       'cat_id_3', 'store_id_1', 'store_id_2', 'store_id_3', 'store_id_4',
       'store_id_5', 'store_id_6', 'store_id_7', 'store_id_8', 'store_id_9',
       'store_id_10', 'state_id_1', 'state_id_2', 'state_id_3', 'item_id_0',
       'item_id_1', 'item_id_2', 'item_id_3', 'item_id_4', 'item_id_5',
       'item_id_6', 'item_id_7', 'item_id_8', 'item_id_9', 'item_id_10',
       'item_id_11', 'item_id_12'],
      dtype='object')

In [10]:
#Review the price data
len(prices_df.columns)

4

In [11]:
prices_df.columns

Index(['store_id', 'item_id', 'wm_yr_wk', 'sell_price'], dtype='object')

In [12]:
#duplicate columns across three tables
# ['store_id', 'item_id', 'wm_yr_wk', 'id']

#Later we will add these features:
# ['total_sale', 'quantity_sold']

### Data Schema

In [13]:
schema = ['id','d', 'day','item_id_#', 'item_id','date', 'sell_price',
          'dept_id', 'cat_id', 'store_id',
          'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 
          
       'state_id', 'dept_id_1', 'dept_id_2', 'dept_id_3', 'dept_id_4',
       'dept_id_5', 'dept_id_6', 'dept_id_7', 'cat_id_1', 'cat_id_2',
       'cat_id_3', 'store_id_1', 'store_id_2', 'store_id_3', 'store_id_4',
       'store_id_5', 'store_id_6', 'store_id_7', 'store_id_8', 'store_id_9',
       'store_id_10', 'state_id_1', 'state_id_2', 'state_id_3', 'item_id_0',
       'item_id_1', 'item_id_2', 'item_id_3', 'item_id_4', 'item_id_5',
       'item_id_6', 'item_id_7', 'item_id_8', 'item_id_9', 'item_id_10',
       'item_id_11', 'item_id_12',
 
  
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 
       'Halloween', 'Christmas','EidAlAdha', 'Eid al-Fitr', 'OrthodoxChristmas',
       'NewYear','Mother\'s day', 'ValentinesDay', 'Father\'s day', 'Thanksgiving',
       'StPatricksDay', 'IndependenceDay', 'MemorialDay', 'PresidentsDay',
       'MartinLutherKingDay', 'VeteransDay', 'LaborDay', 'ColumbusDay',
       'LentStart', 'LentWeek2', 'OrthodoxEaster', 'Chanukah End',
       'Ramadan starts', 'Easter', 'Cinco De Mayo', 'Pesach End', 'Purim End',
       'NBAFinalsStart', 'NBAFinalsEnd', 'SuperBowl', 'Cultural', 'Religious',
       'Sporting', 'National',
         
       'sell_price', 'total_sale', 'quantity_sold']

In [14]:
sales_df.iloc[:,sales_df.columns.get_loc(first_day)]

0        0
1        0
2        0
3        0
4        0
        ..
30485    0
30486    0
30487    0
30488    0
30489    0
Name: d_1, Length: 30490, dtype: int64

In [15]:
len(schema)

96

### Join the Sales_preprocessed and Calendar_preprocessed Tables on Day Column

### Add sales price 

In [16]:
prices_df.columns

Index(['store_id', 'item_id', 'wm_yr_wk', 'sell_price'], dtype='object')

In [17]:
#engineer the unique id's for each item in the price table
prices_df['id'] = prices_df['item_id'] + '_' + prices_df['store_id']+ '_validation'

In [18]:
# Reorganize columns
columns_final = schema.copy()

In [19]:
len(columns_final)

96

## Apply transformation to the entire dataset

#### takes the the daily sales data for each item from the sales file, daily calendar info from calendar file
#### and gets the price info from sell_price file

In [20]:
# Extracting just sales data
items_for_sale = sales_df.iloc[:,:sales_df.columns.get_loc(first_day)]
items_for_sale.columns

Index(['id', 'item_id_#', 'item_id', 'dept_id', 'cat_id', 'store_id',
       'state_id', 'dept_id_1', 'dept_id_2', 'dept_id_3', 'dept_id_4',
       'dept_id_5', 'dept_id_6', 'dept_id_7', 'cat_id_1', 'cat_id_2',
       'cat_id_3', 'store_id_1', 'store_id_2', 'store_id_3', 'store_id_4',
       'store_id_5', 'store_id_6', 'store_id_7', 'store_id_8', 'store_id_9',
       'store_id_10', 'state_id_1', 'state_id_2', 'state_id_3', 'item_id_0',
       'item_id_1', 'item_id_2', 'item_id_3', 'item_id_4', 'item_id_5',
       'item_id_6', 'item_id_7', 'item_id_8', 'item_id_9', 'item_id_10',
       'item_id_11', 'item_id_12'],
      dtype='object')

In [21]:
#  Initiate the  dataframe
df_final = pd.DataFrame()  # This is to store  the final merged data
df_temp = pd.DataFrame()   #To store temporary data

#determine the file path and the name
file_name_prefix = 'temp/in_progress_preprocessed_data'
file_format = '.csv'

In [22]:
# Set packet sizes: each file will have certain number of days.

start_date = int(input("What is the starting day: "))   

end_date = int(input("What is the last day: ")) 


step = 3   # set the number of days to include in each file

packets = list(range(start_date+step,end_date,step)) + [end_date]

indx_processed = [start_date-1]  #list of last day processed in each file

suffix_end = '_' + str(start_date) + '_' 

In [23]:
packets

[4, 7, 10]

## The next step is expensive to run. Make sure to back up the files in the temp folder. This will override previous data.

In [24]:
i = start_date-1  #index of the starting day
print(f'Saving data to "{file_name_prefix}":\n')



for col in calendar_df['d'][start_date-1:end_date]:    # iterate over days in the calendar df
    
        

    
    ### Extracting the day column
    
    day = int(col.split('d_')[1])   #Extract the day number from col being iterated
    #print(f" #Extracted the day number from {col} being iterated")
    
    
    items_for_sale['day'] = day   #Adding the day # as a column
    #print(" #Adding the day # as a column")
    
    
    
    
    
    items_for_sale['quantity_sold'] = sales_df[col]  # Copy the day column from the sales 
    #print("# Copied the day columns from the sales_df")
 
    
    items_sold_day = items_for_sale[ items_for_sale['quantity_sold'] > 0 ]  #filter the items sold on this date
    #print("#filtered the items sold on this date")


    ### merging the calendar information 

    #filtering the calendar info for the day
    calendar_info = calendar_df[calendar_df['d']==col] 
    #print('filtering the calendar info for the day')
    
    
    ### merging the calendar information
    items_sold_day_calendar = items_sold_day.merge(calendar_info, how='left', left_on='day', right_on='day')
    #print('merging calendar info')
    

    ### merging with the selling price 
    temp_df = pd.merge(items_sold_day_calendar,prices_df[['id','wm_yr_wk','sell_price']], how='left', suffixes=['_x', '_y'])
    #print('merging with the selling price')
    
    
    #Calculate the total sale amount in $
    temp_df['total_sale'] =  temp_df.sell_price * temp_df.quantity_sold    
    #print('Calculate the total sale amount in $')
    
    
    #Append the data from the current loop to file
    df_temp = df_temp.append(temp_df[schema])
    #print('Append the data from the current loop to file')  
    

    
    #increment index:'i' by 1 to match the day processed
    i+=1  
    
    ### Getting ready to export to csv
    #### set up the file name 
    
    # packets is the list of indexes when the temp df will be exported to a file
    if (i) in packets: 
        
        #setting up the file name suffix start : (file_name_sfxstart_sfxend)
        if indx_processed[-1] == 0:  
            suffix_start = '_1_'    ## first day is 1, therefore conditional for 0+1
        
        else:    
            suffix_start = '_' + str(indx_processed[-1]+1) + '_'
        
        
        # change i to string to be able to concat with the file name
        suffix_end = str(i)
        
        ### Finalize the file name
        file_name_suffix = suffix_start + suffix_end  
        
        ### combine with the data format
        path = file_name_prefix + file_name_suffix + file_format
        
        df_temp.to_csv(path, mode='a', header=True, index_label=0)   #Append the data from the current loop to file
        
        print(f'Processed Day_{day}')
        print(f"\nsaved file:  {path}")
        
        
        #Save the last day processed to index variable
        indx_processed.append(i)
        
        #Restart the df_final to free up memory
        df_temp = pd.DataFrame()

Saving data to "temp/in_progress_preprocessed_data":

Processed Day_1
Processed Day_2
Processed Day_3

saved file:  temp/in_progress_preprocessed_data_1_4.csv
Processed Day_4
Processed Day_5
Processed Day_6

saved file:  temp/in_progress_preprocessed_data_5_7.csv
Processed Day_7
Processed Day_8
Processed Day_9

saved file:  temp/in_progress_preprocessed_data_8_10.csv
Processed Day_10


# Finished preprocessing.